## Predict

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mmocr.apis import TextRecInferencer, TextDetInferencer, MMOCRInferencer
import os
import sys
import cv2
from time import sleep
import time
import gc

# def image_rotate(image, position, angle):
#     rotation_matrix = cv2.getRotationMatrix2D(position, angle, 1)
#     # rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
#     # rotated_image = cv2.transpose(image)
#     rotated_image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)

#     rotated_position = np.dot(rotation_matrix[:, :2], np.array(position)) + rotation_matrix[:, 2]

#     h, w = rotated_image.shape[:2]
#     image = rotated_image[int(rotated_position[1] - h/2):int(rotated_position[1] + h/2), int(rotated_position[0] - w/2):int(rotated_position[0] + w/2)]
#     return image

def image_rotate(image, position, angle):
    rotation_matrix = cv2.getRotationMatrix2D(position, angle, 1)
    
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    rotated_position = np.dot(rotation_matrix[:, :2], np.array(position)) + rotation_matrix[:, 2]

    return rotated_image

def adjust_contrast_adaptive_histogram(image, clip_limit=2.0, tile_grid_size=(15, 15)):
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    adjusted_image = clahe.apply(image)
    return adjusted_image

def plotimg (img, filename):
    plt.figure(figsize=(5, 10))
    plt.imshow(img) #result['visualization'][0]
    plt.title(filename)
    plt.show()

def mkdir (path):
    try:
        os.mkdir(path)
    except:
        pass

def draw_text_based_on_direction(image, points, text, font_scale=1, font_thickness=1, font_color=(255, 255, 255)):
    # 座標轉換為OpenCV的格式
    points = np.array(points, np.int32).reshape((-1, 1, 2))

    # Polyline的長度
    x_length = np.abs(points[:, 0, 0].max() - points[:, 0, 0].min())
    y_length = np.abs(points[:, 0, 1].max() - points[:, 0, 1].min())

    # 標註文字的位置和方向
    # position = (points[:, 0, 0].mean(), points[:, 0, 1].mean())
    direction = 'horizontal' if x_length >= y_length else 'vertical'

    angle = 0 if direction == 'horizontal' else 90

    cv2.drawContours(image, [points], -1, (0, 255, 0), 3)

    if angle != 0:
        # rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
        # rotated_image = cv2.transpose(image)
        rotated_image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)

        # 重新計算，旋轉標註文字位置
        # rotated_position = np.dot(rotation_matrix[:, :2], np.array(position)) + rotation_matrix[:, 2]

        # h, w = rotated_image.shape[:2]
        # plt.imshow(rotated_image)
        # plt.show()
        # print(h, w)
        image = rotated_image
        # image = rotated_image[int(rotated_position[1] - h/2):int(rotated_position[1] + h/2), int(rotated_position[0] - w/2):int(rotated_position[0] + w/2)]
        print(points.shape, type(points))
        rotated_points = np.array([[point[0][1], (image.shape[0] - point[0][0])] for point in points] , np.int32).reshape((-1,1,2))
        print(rotated_points.shape)
        center_x = image.shape[1] // 2
        center_y = image.shape[0] // 2

        distances = np.linalg.norm(rotated_points - [center_x, center_y], axis=2)
        closet_point_index = np.argmin(distances)

        position = tuple(rotated_points[closet_point_index][0])
        
    else:
        center_x = image.shape[1] // 2
        center_y = image.shape[0] // 2

        distances = np.linalg.norm(points - [center_x, center_y], axis=2)
        closet_point_index = np.argmin(distances)

        position = tuple(points[closet_point_index][0])
        # rotated_position = position

    cv2.putText(image, text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_color, font_thickness, cv2.LINE_AA)

    if angle != 0:
        # image = image_rotate(image, position, angle)
        # image = image_rotate(image, position, angle)
        # image = image_rotate(image, position, angle)
        image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        # image = cv2.transpose(image)
        # image = cv2.transpose(image)
        # image = cv2.transpose(image)
        # reverse_rotation_matrix = cv2.getRotationMatrix2D(rotated_position, -angle, 1)
        # image = cv2.warpAffine(image, reverse_rotation_matrix, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)

    return image

# path = "./data/icdar2015/imgs/training"
path = "./data/imgs/CLAHE" # equalization histogram image
imgs = [file for file in os.listdir(path) if file.find(".jpg") != -1]
for filename in imgs:
    imgpath = os.path.join(path, filename)
    img = cv2.imread(os.path.join("./data/imgs", filename), cv2.IMREAD_COLOR) #imgpath
    # checkpoint = "./data/icdar/epoch_60.pth"
    # cfg_file = "./data/icdar/dbnet_resnet50-dcnv2_fpnc_1200e_icdar2015.py"
    det_checkpoint = os.path.join(os.getcwd(), "by慶偉/model/epoch_60.pth")
    det_file = os.path.join(os.getcwd(), "by慶偉/model/dbnetpp_resnet50-dcnv2_fpnc_1200e_icdar2015_plusctw1500.py")
    
    rec_checkpoint = "abinet_union14m-cbf19742.pth"
    # rec_checkpoint = "abinet_pretrain-45deac15.pth"
    rec_file = os.path.join(os.getcwd(), "mmocr/configs/textrecog/abinet/abinet-vision_10e_union14m.py")
    # rec_file = os.path.join(os.getcwd(), "mmocr/configs/textrecog/abinet/abinet_20e_st-an_mj.py")
    model = MMOCRInferencer(det=det_file, det_weights=det_checkpoint, 
                            rec=rec_file, rec_weights=rec_checkpoint) 

    # infer = TextDetInferencer(cfg_file, checkpoint)



    result = model(imgpath, return_vis=True)

    texts = result['predictions'][0]['rec_texts']
    polygons = result['predictions'][0]['det_polygons']
    scores = result['predictions'][0]['rec_scores']
    for idx, polygon in enumerate(polygons):
        text = texts[idx]
        
        cnt1 = [int(value) for value in polygon]
        cnt1 = np.array([cnt1[i:i+2] for i in range(0, len(cnt1), 2)])
        area = cv2.contourArea(cnt1)

        if (area > 10000 and len(text) > 2): #(scores[idx] > 0.5):
        
            # cv2.drawContours(img, [cnt1], -1, (0, 255, 0), 3)
            
            # Retrieve bounding box of cnt1
            # x, y, w, h = cv2.boundingRect(cnt1)
            # replace any '<UKN>'  as ''
            text = text.replace('<UKN>', '').upper()
            # puttext the text on x,y
            img = draw_text_based_on_direction(img, cnt1, text, font_scale=5, font_thickness=10, font_color=(0, 255, 0))

            # cv2.putText(img, text, (x-100, y), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 255, 0), 10, cv2.LINE_AA)

        print(f"result : {texts}")
        gc.collect()
    if (filename.find("img") == -1):
        cv2.imwrite(f"C:/Users/User/Desktop/C_VSCode/mmocr/data/imgs/dst/{filename}", img)

        # print(result['predictions'][0])
        
        # plt.figure(figsize=(5, 10))
        # plt.imshow(img) #result['visualization'][0]
        # plt.title(filename)
        # plt.show()
    
    gc.collect()
    # sys.exit()

## Plot training curve

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import ast
import matplotlib.pyplot as plt
from time import time, sleep


path = os.getcwd()
path = os.path.join(path, "mmocr")
path = os.path.join(path, "tools")
path = os.path.join(path, "data")
path = os.path.join(path, "icdar2015")
path = os.path.join(path, "20240308_143720")
path = os.path.join(path, "vis_data")
jsonfile = [file for file in os.listdir(path) if file.find(".json") != -1 and file.find("scalar") == -1][0]
js_path = os.path.join(path, jsonfile)

file = []
try:
    with open(js_path, 'r', encoding='utf-8') as f:
        content = f.read()
        # print(content)
        lines = content.strip().split('\n')


        dictionaries = []
        for line in lines:
            # print(line)
            line.replace("NaN", "0")
            try:
                # Use ast.literal_eval instead of eval for safer evaluation
                dictionary = ast.literal_eval(line)
                # Convert values to float, replacing '0' with 0
                dictionary = {k: float(v) if v != '0' else 0 for k, v in dictionary.items()}
                dictionaries.append(dictionary)
            except (SyntaxError, ValueError) as e:
                print(f"Error evaluating line: {line}\nError: {e}")

        # dictionaries = [eval(line) for line in lines]
        # print(dictionaries)

        # try:
        #     dictionary = eval(content)
        #     print(dictionary)
        # except Exception as e:
        #     print(f"Error: {e}")
        # js = json.loads(content)
        # file.append(js)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    # Handle the exception or exit the program if necessary

# Now, you can proceed with the normalization if js is defined successfully
df = pd.DataFrame(dictionaries)

# iterations = [entry["iter"] for entry in dictionaries]
iterations = (df["iter"].values[:])
# loss_values = [entry["loss"] for entry in dictionaries]
loss_values = df["loss"].values[:]
# loss_prob_values = [entry["loss_prob"] for entry in dictionaries]
loss_prob_values = df["loss_prob"].values[:]

# Plot the loss values against iterations
plt.plot(iterations, loss_values, marker='o', linestyle='-', color='b', label='Loss')
plt.plot(iterations, loss_prob_values, marker='s', linestyle='-', color='r', label='Loss Prob')

plt.title('Loss Growth Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.close()


df.to_csv("training_log.csv", index=False)
df

## 查看code

### mmocr fine tuning by icdar dataset

In [ ]:
import mmcv
from mmocr.apis import MMOCRInferencer
import torch as tc



ocr = MMOCRInferencer(det='DBNet', rec='CRNN')
ocr('./icdar2015/ch4_training_images/img_1.jpg', show=False, print_result=True)
# ocr('./dataset/0.bmp', show=False, print_result=True)



### ddnet ++

In [ ]:
_base_ = [
    '_base_dbnetpp_resnet50-dcnv2_fpnc.py',
    '../_base_/pretrain_runtime.py',
    '../_base_/datasets/synthtext.py',
    '../_base_/schedules/schedule_sgd_100k.py',
]

train_pipeline = [
    dict(type='LoadImageFromFile', color_type='color_ignore_orientation'),
    dict(
        type='LoadOCRAnnotations',
        with_bbox=True,
        with_polygon=True,
        with_label=True,
    ),
    dict(type='FixInvalidPolygon'),
    dict(
        type='TorchVisionWrapper',
        op='ColorJitter',
        brightness=32.0 / 255,
        saturation=0.5),
    dict(
        type='ImgAugWrapper',
        args=[['Fliplr', 0.5],
              dict(cls='Affine', rotate=[-10, 10]), ['Resize', [0.5, 3.0]]]),
    dict(type='RandomCrop', min_side_ratio=0.1),
    dict(type='Resize', scale=(640, 640), keep_ratio=True),
    dict(type='Pad', size=(640, 640)),
    dict(
        type='PackTextDetInputs',
        meta_keys=('img_path', 'ori_shape', 'img_shape'))
]

synthtext_textdet_train = _base_.synthtext_textdet_train
synthtext_textdet_train.pipeline = train_pipeline

train_dataloader = dict(
    batch_size=16,
    num_workers=8,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    dataset=synthtext_textdet_train)

auto_scale_lr = dict(base_batch_size=16)

### fpn ResNET

In [ ]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
import torch


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3mb4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=7, scale=1):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        # self.avgpool = nn.AvgPool2d(7, stride=1)
        # self.fc = nn.Linear(512 * block.expansion, num_classes)

        # Top layer
        self.toplayer = nn.Conv2d(2048, 256, kernel_size=1, stride=1, padding=0)  # Reduce channels
        self.toplayer_bn = nn.BatchNorm2d(256)
        self.toplayer_relu = nn.ReLU(inplace=True)

        # Smooth layers
        self.smooth1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.smooth1_bn = nn.BatchNorm2d(256)
        self.smooth1_relu = nn.ReLU(inplace=True)

        self.smooth2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.smooth2_bn = nn.BatchNorm2d(256)
        self.smooth2_relu = nn.ReLU(inplace=True)

        self.smooth3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.smooth3_bn = nn.BatchNorm2d(256)
        self.smooth3_relu = nn.ReLU(inplace=True)

        # Lateral layers
        self.latlayer1 = nn.Conv2d(1024, 256, kernel_size=1, stride=1, padding=0)
        self.latlayer1_bn = nn.BatchNorm2d(256)
        self.latlayer1_relu = nn.ReLU(inplace=True)

        self.latlayer2 = nn.Conv2d(512,  256, kernel_size=1, stride=1, padding=0)
        self.latlayer2_bn = nn.BatchNorm2d(256)
        self.latlayer2_relu = nn.ReLU(inplace=True)

        self.latlayer3 = nn.Conv2d(256,  256, kernel_size=1, stride=1, padding=0)
        self.latlayer3_bn = nn.BatchNorm2d(256)
        self.latlayer3_relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(1024, 256, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(256, num_classes, kernel_size=1, stride=1, padding=0)

        self.scale = scale
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _upsample(self, x, y, scale=1):
        _, _, H, W = y.size()
        return F.upsample(x, size=(H // scale, W // scale), mode='bilinear')

    def _upsample_add(self, x, y):
        _, _, H, W = y.size()
        return F.upsample(x, size=(H, W), mode='bilinear') + y

    def forward(self, x):
        h = x
        h = self.conv1(h)
        h = self.bn1(h)
        h = self.relu1(h)
        h = self.maxpool(h)

        h = self.layer1(h)
        c2 = h
        h = self.layer2(h)
        c3 = h
        h = self.layer3(h)
        c4 = h
        h = self.layer4(h)
        c5 = h

        # Top-down
        p5 = self.toplayer(c5)
        p5 = self.toplayer_relu(self.toplayer_bn(p5))

        c4 = self.latlayer1(c4)
        c4 = self.latlayer1_relu(self.latlayer1_bn(c4))
        p4 = self._upsample_add(p5, c4)
        p4 = self.smooth1(p4)
        p4 = self.smooth1_relu(self.smooth1_bn(p4))

        c3 = self.latlayer2(c3)
        c3 = self.latlayer2_relu(self.latlayer2_bn(c3))
        p3 = self._upsample_add(p4, c3)
        p3 = self.smooth2(p3)
        p3 = self.smooth2_relu(self.smooth2_bn(p3))        

        c2 = self.latlayer3(c2)
        c2 = self.latlayer3_relu(self.latlayer3_bn(c2))
        p2 = self._upsample_add(p3, c2)
        p2 = self.smooth3(p2)
        p2 = self.smooth3_relu(self.smooth3_bn(p2))

        p3 = self._upsample(p3, p2)
        p4 = self._upsample(p4, p2)
        p5 = self._upsample(p5, p2)

        out = torch.cat((p2, p3, p4, p5), 1)
        out = self.conv2(out)
        out = self.relu2(self.bn2(out))
        out = self.conv3(out)
        out = self._upsample(out, x, scale=self.scale)

        return out


def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model


def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        pretrained_model = model_zoo.load_url(model_urls['resnet50'])
        state = model.state_dict()
        for key in state.keys():
            if key in pretrained_model.keys():
                state[key] = pretrained_model[key]
        model.load_state_dict(state)
    return model


def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        pretrained_model = model_zoo.load_url(model_urls['resnet101'])
        state = model.state_dict()
        for key in state.keys():
            if key in pretrained_model.keys():
                state[key] = pretrained_model[key]
        model.load_state_dict(state)
    return model

def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        pretrained_model = model_zoo.load_url(model_urls['resnet152'])
        state = model.state_dict()
        for key in state.keys():
            if key in pretrained_model.keys():
                state[key] = pretrained_model[key]
        model.load_state_dict(state)
    return model

##### 資料後處理 EqualHist (CLAHE)

In [ ]:
import os
import cv2
import sys
import matplotlib.pyplot as plt

def adjust_contrast_adaptive_histogram(image, clip_limit=2.0, tile_grid_size=(15, 15)):
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    adjusted_image = clahe.apply(image)
    return adjusted_image

srcpath = os.path.join(os.getcwd(), "data/imgs")
dstpath = os.path.join(srcpath, "CLAHE")
imgs = [file for file in os.listdir(srcpath) if file.find(".jpg") != -1]

for filename in imgs:
    # img = cv2.imread(os.path.join(srcpath, filename))
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # dst = cv2.equalizeHist(gray)
    # plt.imshow(dst)
    # plt.show()
    # sys.exit()
    image_path = os.path.join(srcpath, filename)
    original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load the image in grayscale

    cv2.medianBlur(original_image, 21, original_image)

    equalized_image = adjust_contrast_adaptive_histogram(original_image, 9, (7, 7))

    equalized_image = cv2.bilateralFilter(equalized_image, 13, 75, 75)

    # Apply histogram equalization
    # equalized_image = cv2.equalizeHist(original_image)

    # Convert the original and equalized images to RGB for display
    original_rgb = cv2.cvtColor(original_image, cv2.COLOR_GRAY2RGB)
    equalized_rgb = cv2.cvtColor(equalized_image, cv2.COLOR_GRAY2RGB)
    
    # adjusted_image = cv2.convertScaleAbs(equalized_rgb, alpha=0.7, beta=0)
    if (filename.find("12.jpg") != -1):
        # Display the original and equalized images side by side
        plt.figure(figsize=(10, 4))

        plt.subplot(1, 2, 1)
        plt.title('Original Image')
        plt.imshow(original_rgb)
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title('Equalized Image')
        plt.imshow(equalized_rgb)
        plt.axis('off')

        plt.show()
        
        # plt.imshow(adjusted_image)
        # plt.show()

    cv2.imwrite(os.path.join(dstpath, filename), equalized_rgb)
    # cv2.imwrite(os.path.join(dstpath, filename), equalized_rgb)
    # cv2.imwrite(os.path.join(os.path.join(dstpath, "constrast"), filename), adjusted_image)